In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_google_genai import ChatGoogleGenerativeAI
from tqdm import tqdm
from typing import TypedDict, Annotated
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import HumanMessage, BaseMessage

: 

In [ ]:
load_dotenv()

In [ ]:
llm = ChatGoogleGenerativeAI(
    model = 'gemini-2.5-flash'
)

In [ ]:
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages] 

In [ ]:
def chatNode(state: ChatState):
    messages = state['messages']
    
    response = llm.invoke(messages)

    return {'messages' : [response]}

In [ ]:
checkpointer = InMemorySaver()

graph = StateGraph(ChatState)

graph.add_node('chat_node', chatNode)

graph.add_edge(START, 'chat_node')
graph.add_edge('chat_node', END)

chat = graph.compile(checkpointer=checkpointer)


In [ ]:
thread_id = '1'

while True:

    user_message = input("Type Here : ")

    print('User : ', user_message)

    config = {
        'configurable': {
            'thread_id': thread_id
        }
    }

    if user_message.strip().lower() in ['exit', 'quit', 'bye']:
        break 

    response = chat.invoke({'messages': [HumanMessage(content=user_message)]}, config=config)
    
    print('AI : ', response['messages'][-1].content)